[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/CCSR-colab/blob/main/CCSR_gradio_colab.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/CCSR
%cd /content/CCSR

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CCSR/resolve/main/real-world_ccsr.ckpt -d /content/CCSR/weights -o real-world_ccsr.ckpt

!pip install -q einops pytorch-lightning omegaconf open-clip-torch gradio==3.50.2
!pip install -q https://download.pytorch.org/whl/cu121/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl
!pip install -q -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers

!rm -rf /content/CCSR/preset/test_datasets/19.jpg
!mv /content/CCSR/preset/test_datasets/49.jpg /content/CCSR/preset/test_datasets/input.jpg

import gradio as gr
import subprocess

def generate(image, prompt, seed):
    print(image, prompt, seed)
    command = f"python inference_ccsr.py --input preset/test_datasets --config configs/model/ccsr_stage2.yaml --ckpt weights/real-world_ccsr.ckpt --steps 45 --sr_scale 4 --t_max 0.6667 --t_min 0.3333 --color_fix_type adain --output experiments/test --device cuda --repeat_times 1"
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("Output:", result.stdout)
        output_path = '/content/CCSR/experiments/test/input.jpg'
        return output_path
    except subprocess.CalledProcessError as e:
        print("Error:", e.stderr)
        return None

with gr.Blocks() as demo:
    with gr.Row():
      with gr.Column():
          image = gr.Image(type='filepath')
          textbox = gr.Textbox(show_label=False, value="a person facing the camera, making a hand gesture, indoor")
          seed = gr.Slider(minimum=0, maximum=1000000, value=643534)
          button = gr.Button()
      output_image = gr.Image(show_label=False, type="filepath", interactive=False, height=512, width=512)
    button.click(fn=generate, inputs=[image, textbox, seed], outputs=[output_image])

demo.queue().launch(inline=False, share=True, debug=True)